In [1]:
from huggingface_hub import notebook_login
from datasets import load_dataset, ClassLabel
from IPython.display import display, HTML
import random
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

Main Language Modeling:

- Causal Language Modeling : Model predicts the next token in the given sentence. So the targets are **inputs shifted to right.**

- Masked Language Modeling: Model has to predict some masked tokens in the given sentence, and has access to entire sentence

In [2]:
# One training step will be to change this dataset with another
wikiset = load_dataset('wikitext', 'wikitext-2-raw-v1')

# datasets = load_dataset("text", data_files={"train": path_to_train.txt,"validation": path_to_validation.txt}


Generating test split:   0%|          | 0/4358 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/36718 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3760 [00:00<?, ? examples/s]

Dataset wikitext downloaded and prepared to /root/.cache/huggingface/datasets/wikitext/wikitext-2-raw-v1/1.0.0/a241db52902eaf2c6aa732210bead40c090019a499ceb13bcbfa3f8ab646a126. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [6]:
wikiset['train'][56]

{'text': ' = = Construction = = \n'}

In [12]:
len(wikiset['train'])

36718

In [3]:
# Writing a function to show random elements

def show_random_elements(dataset, num_examples=5):
    assert num_examples <= len(dataset)  # dataset must have sufficient data
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset) - 1)
        while pick in picks:
            pick = random.randint(0, len(dataset) - 1)
        picks.append(pick)
        
    df = pd.DataFrame(dataset[picks])
    display(HTML(df.to_html()))

In [16]:
show_random_elements(wikiset['train'])

,text
0,"If G is a DAG , its transitive closure is the graph with the most edges that represents the same reachability relation . It has an edge u → v whenever u can reach v. That is , it has an edge for every related pair u ≤ v of distinct elements in the reachability relation of G , and may therefore be thought of as a direct translation of the reachability relation ≤ into graph @-@ theoretic terms . The same method of translating partial orders into DAGs works more generally : for every finite partially ordered set ( S , ≤ ) , the graph that has a vertex for each member of S and an edge for each pair of elements related by u ≤ v is automatically a transitively closed DAG , and has ( S , ≤ ) as its reachability relation . In this way , every finite partially ordered set can be represented as the reachability relation of a DAG . \n"
1,"In Swiss system tournaments , the tournament director tries to ensure that each player receives , as nearly as possible , the same number of games as White and Black , and that the player 's color alternates from round to round . After the first round , the director may deviate from the otherwise prescribed pairings in order to give as many players as possible their equalizing or due colors . More substantial deviations are permissible to avoid giving a player two more blacks than whites ( for example , three blacks in four games ) than vice versa , since extra whites "" cause far less player distress "" than extra blacks , which impose "" a significant handicap "" on the affected player . Tournaments with an even number of rounds cause the most problems , since if there is a disparity , it is greater ( e.g. , a player receiving two whites and four blacks ) . \n"
2,
3,= = = Ratings = = = \n
4,= = = Ely Link Road = = = \n


we are going to take **all the texts** in our dataset and **concatenate them** after they are tokenized. Then we will split them in examples of a certain sequence length. This way the model will receive chunks of contiguous text.

The labels will be the same as the inputs, shifted to the left.

In [4]:
model_cp = "gpt2"
tokenizer_cp = "sgugger/gpt2-like-tokenizer"

from transformers import AutoTokenizer, AutoModelForCausalLM, AutoConfig
from transformers import Trainer, TrainingArguments

tokeniser = AutoTokenizer.from_pretrained(tokenizer_cp)

2024-02-02 01:06:17.846703: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-02 01:06:17.846823: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-02 01:06:18.106853: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


tokenizer_config.json:   0%|          | 0.00/236 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/396k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/678k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/90.0 [00:00<?, ?B/s]

In [5]:
def tokenise_text(examples):
    return tokeniser(examples['text'])

In [6]:
tokenised_wiki = wikiset.map(tokenise_text,
                             batched=True,
                             num_proc=4,
                             remove_columns=['text'])  # will return just ids / masks

#0:   0%|          | 0/2 [00:00<?, ?ba/s]

#1:   0%|          | 0/2 [00:00<?, ?ba/s]

#2:   0%|          | 0/2 [00:00<?, ?ba/s]

#3:   0%|          | 0/2 [00:00<?, ?ba/s]

#0:   0%|          | 0/10 [00:00<?, ?ba/s]

...

#3:   0%|          | 0/10 [00:00<?, ?ba/s]

#0:   0%|          | 0/1 [00:00<?, ?ba/s]

#1:   0%|          | 0/1 [00:00<?, ?ba/s]

#2:   0%|          | 0/1 [00:00<?, ?ba/s]

#3:   0%|          | 0/1 [00:00<?, ?ba/s]

In [20]:
tokenised_wiki

DatasetDict({
    test: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 4358
    })
    train: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 36718
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 3760
    })
})

In [21]:
show_random_elements(tokenised_wiki['train'])

,input_ids,attention_mask
0,"[261, 1134, 301, 4183, 10383, 307, 196, 16713, 225, 3503, 307, 196, 2436, 201, 225, 6983, 390, 5825, 307, 8081, 201, 867, 6558, 3315, 201, 7814, 225, 5559, 3315, 209, 261, 15134, 13120, 1085, 12470, 225, 224, 23648, 505, 227, 7668, 444, 1069, 201, 18306, 196, 845, 218, 3438, 218, 10472, 587, 444, 7650, 209, 240, 1939, 3158, 72, 343, 22566, 1552, 1134, 994, 261, 16818, 3725, 529, 621, 15028, 319, 766, 2855, 201, 281, 10472, 708, 198, 906, 3238, 3473, 209, 252]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]"
1,"[238, 238, 238, 22438, 225, 7125, 238, 238, 238, 252]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]"
2,"[4346, 252]","[1, 1]"
3,"[238, 238, 2392, 6191, 238, 238, 252]","[1, 1, 1, 1, 1, 1, 1]"
4,"[2496, 3785, 2598, 268, 83, 1753, 277, 3037, 469, 10598, 201, 4545, 1623, 959, 457, 782, 7269, 218, 198, 674, 319, 457, 1678, 277, 2459, 1248, 550, 196, 1311, 3943, 1063, 209, 261, 4545, 1623, 268, 83, 782, 5388, 218, 198, 674, 258, 604, 1991, 702, 457, 455, 201, 445, 281, 806, 201, 198, 12535, 6489, 1863, 227, 196, 17513, 806, 24377, 2032, 12486, 587, 2598, 3833, 201, 225, 1623, 258, 2636, 227, 10658, 198, 2006, 881, 227, 3785, 2598, 209, 261, 7771, 478, 6489, 959, 630, 319, 457, 1802, 277, 2459, 1248, 445, 1983, 697, 225, 573, 225, 5295, ...]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...]"


In [36]:
data_test = [[1, 2, 3,], [2, 5, 7], [5, 6, 8]]
# b = 0
# print(data_test[b] + data_test[b+1])

# x = [sum(data_test[b], []) for b in range(3)]  # [1, 2, 5]
sum(data_test[0], [])  # why this is not working??

TypeError: can only concatenate list (not "int") to list

In [48]:
test_data = tokenised_wiki['train'][:3]
test_data

{'input_ids': [[], [238, 8576, 9441, 2987, 238, 252], []],
 'attention_mask': [[], [1, 1, 1, 1, 1, 1], []]}

In [49]:
# test_data.keys() # dict_keys(['input_ids', 'attention_mask'])

concat_examples = {b: sum(test_data[b], []) for b in test_data.keys()}
concat_examples

{'input_ids': [238, 8576, 9441, 2987, 238, 252],
 'attention_mask': [1, 1, 1, 1, 1, 1]}

In [50]:
concat_examples[list(test_data.keys())[0]]  # [238, 8576, 9441, 2987, 252]


[238, 8576, 9441, 2987, 238, 252]

In [51]:
test_data.items()

dict_items([('input_ids', [[], [238, 8576, 9441, 2987, 238, 252], []]), ('attention_mask', [[], [1, 1, 1, 1, 1, 1], []])])

In [7]:
print(tokeniser.model_max_length)  # 1024 is length the model is trained on
# block_size = tokenizer.model_max_length
block_size = 128

1024


In [8]:
def group_texts(examples):
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    # above will make sub_list inside input_ids and attention_masks to be extended
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # provide the total_length of input_ids
    # we drop the remainder of the sentence, like truncating
    tot_length = (total_length // block_size) * block_size  
    # // will porvide only the whole number after dividing
    # (3268 // 128) * 128 
    # split by chunks of max_len
    result = {
        k: [t[i : i + block_size] for i in range(0, tot_length, block_size)]
            for k, t in concatenated_examples.items()
    }
    # k will be keys while t will be values
    result["labels"] = result["input_ids"].copy() # inputs are duplicated as labels
    # transformers library will apply the "shifting to right"
    return result

In [9]:
lm_datasets = tokenised_wiki.map(
    group_texts,
    batched=True,
    batch_size=1000,
    num_proc=6
)

#0:   0%|          | 0/1 [00:00<?, ?ba/s]

#1:   0%|          | 0/1 [00:00<?, ?ba/s]

#2:   0%|          | 0/1 [00:00<?, ?ba/s]

#5:   0%|          | 0/1 [00:00<?, ?ba/s]

#4:   0%|          | 0/1 [00:00<?, ?ba/s]

...

#1:   0%|          | 0/1 [00:00<?, ?ba/s]

#2:   0%|          | 0/1 [00:00<?, ?ba/s]

#3:   0%|          | 0/1 [00:00<?, ?ba/s]

#4:   0%|          | 0/1 [00:00<?, ?ba/s]

#5:   0%|          | 0/1 [00:00<?, ?ba/s]

In [62]:
tokeniser.decode(lm_datasets['train'][1]['input_ids'])

2024-02-02 00:41:52.384527: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-02 00:41:52.384653: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-02 00:41:52.526201: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


' the " Nameless ", a penal military unit serving the nation of Gallia during the Second Europan War who perform secret black operations and are pitted against the Imperial unit " Calamaty Raven ". \n The game began development in 2010, carrying over a large portion of the work done on Valkyria Chronicles II. While it retained the standard features of the series, it also underwent multiple adjustments, such as making the game more forgiving for series newcomers. Character designer Raita Honjou and composer Hitoshi Sakimoto both returned from previous entries, along with Valkyria Chronicles II director Takeshi Ozawa. A large'

In [10]:
# bring in the config and the model


config = AutoConfig.from_pretrained(model_cp)
model = AutoModelForCausalLM.from_config(config) # this will pull the config only

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

In [69]:
model.get_memory_footprint()

510342192

In [67]:
model_test = AutoModelForCausalLM.from_pretrained(model_cp) # this will load the model
del model_test

In [11]:
notebook_login()

In [15]:
# training args

t_args = TrainingArguments(
    f"{model_cp}-wiki",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    weight_decay=0.01,
    push_to_hub=True,
    hub_model_id=f"kamaljp/{model_cp}-wiki",  # this repo must be available
    report_to="none"
)

In [16]:
# initiate trainer

trainer = Trainer(
    model=model,
    args=t_args,
    train_dataset=lm_datasets['train'],
    eval_dataset=lm_datasets['validation']
) 

In [17]:
trainer.train()

Epoch,Training Loss,Validation Loss
1,6.767800,6.638093
2,6.420200,6.386584
3,6.248600,6.308789


TrainOutput(global_step=3375, training_loss=6.594418258101852, metrics={'train_runtime': 1507.2266, 'train_samples_per_second': 35.813, 'train_steps_per_second': 2.239, 'total_flos': 3526070648832000.0, 'train_loss': 6.594418258101852, 'epoch': 3.0})

In [19]:
import math
eval_results = trainer.evaluate()

NameError: name 'eval_result' is not defined

In [20]:
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

Perplexity: 549.38


In [21]:
trainer.push_to_hub()

model.safetensors:   0%|          | 0.00/498M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Kamaljp/gpt2-wiki/commit/991f8b5abc3f5bca0970a6d7f349804b7fcf9fcc', commit_message='End of training', commit_description='', oid='991f8b5abc3f5bca0970a6d7f349804b7fcf9fcc', pr_url=None, pr_revision=None, pr_num=None)

### Masked Language Modelling

Masked Language Modeling (MLM) we are going to use the same preprocessing as before for our dataset with one additional step: we will randomly mask some tokens (by replacing them by [MASK]) and the labels will be adjusted to only include the masked tokens (we don't have to predict the non-masked tokens).

**The random masking is done with DataCollator at the end.**

In [27]:
mask_model_cp = "bert-base-cased"
tokeniser_bert_cp = "sgugger/bert-like-tokenizer"

tokeniser_bert = AutoTokenizer.from_pretrained(tokeniser_bert_cp)

def tokenise_text(examples):
    return tokeniser_bert(examples['text'])

In [28]:
tokenise_text(wikiset['train'][1])

{'input_ids': [2, 33, 9773, 10627, 4171, 33, 3], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1]}

In [29]:
bert_tokenised_ds = wikiset.map(tokenise_text,  # this has to be a function
                               batched=True,
                               num_proc=4,
                               remove_columns=["text"])

#0:   0%|          | 0/2 [00:00<?, ?ba/s]

#1:   0%|          | 0/2 [00:00<?, ?ba/s]

#3:   0%|          | 0/2 [00:00<?, ?ba/s]

#2:   0%|          | 0/2 [00:00<?, ?ba/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (571 > 512). Running this sequence through the model will result in indexing errors


...

Token indices sequence length is longer than the specified maximum sequence length for this model (554 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (522 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (657 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (514 > 512). Running this sequence through the model will result in indexing errors


#0:   0%|          | 0/1 [00:00<?, ?ba/s]

#1:   0%|          | 0/1 [00:00<?, ?ba/s]

#2:   0%|          | 0/1 [00:00<?, ?ba/s]

#3:   0%|          | 0/1 [00:00<?, ?ba/s]

In [30]:
lm_bert_ds = bert_tokenised_ds.map(group_texts,
                                   batched=True,
                                   batch_size=1000,
                                   num_proc=4)

#0:   0%|          | 0/2 [00:00<?, ?ba/s]

#1:   0%|          | 0/2 [00:00<?, ?ba/s]

#2:   0%|          | 0/2 [00:00<?, ?ba/s]

#3:   0%|          | 0/2 [00:00<?, ?ba/s]

#1:   0%|          | 0/10 [00:00<?, ?ba/s]

...

#3:   0%|          | 0/10 [00:00<?, ?ba/s]

#1:   0%|          | 0/1 [00:00<?, ?ba/s]

#0:   0%|          | 0/1 [00:00<?, ?ba/s]

#2:   0%|          | 0/1 [00:00<?, ?ba/s]

#3:   0%|          | 0/1 [00:00<?, ?ba/s]

In [32]:
from transformers import AutoModelForMaskedLM

bert_config = AutoConfig.from_pretrained(mask_model_cp)
model = AutoModelForMaskedLM.from_config(bert_config)

In [39]:
# training args
tbert_args = TrainingArguments(
    "test-clm",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    weight_decay=0.01,
    push_to_hub=True,
    hub_model_id=f"Kamaljp/{mask_model_cp}-wiki",
    report_to="none",
    num_train_epochs=1)

The data_collator is a function that is responsible of taking the samples and batching them in tensors. 

Here we want to do the random-masking. We could do it as a pre-processing step (like the tokenization) but then the tokens would always be masked the same way at each epoch. 

By doing this step inside the data_collator, we ensure this **random masking is done** in a new way each time we go over the data.

In [40]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(tokenizer=tokeniser_bert,
                                               mlm_probability=0.15)

In [41]:
# trainer takes the additional DataCollator

trainer_bert = Trainer(
    model=model,
    args=tbert_args,
    train_dataset=lm_bert_ds['train'],
    eval_dataset=lm_bert_ds['validation'],
    data_collator=data_collator
)

In [42]:
trainer_bert.train()

Epoch,Training Loss,Validation Loss
1,7.213500,7.200663


TrainOutput(global_step=1173, training_loss=7.4592584868526215, metrics={'train_runtime': 421.4989, 'train_samples_per_second': 44.51, 'train_steps_per_second': 2.783, 'total_flos': 1234474385943552.0, 'train_loss': 7.4592584868526215, 'epoch': 1.0})

In [43]:
eval_results = trainer.evaluate()
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

Perplexity: 549.38


In [44]:
trainer_bert.push_to_hub()

model.safetensors:   0%|          | 0.00/433M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Kamaljp/bert-base-cased-wiki/commit/3087e13379cccef60e6ed884aee222a258c0e4c4', commit_message='End of training', commit_description='', oid='3087e13379cccef60e6ed884aee222a258c0e4c4', pr_url=None, pr_revision=None, pr_num=None)